# Mike Babb
# babbm@uw.edu
# Introduction to Python Part 02

In [ ]:
# Problem Definition

In [ ]:
# standard libraries - installed by default
import csv
from itertools import permutations, combinations
import math
import os
import time

In [ ]:
# external libraries - not installed by default
import numpy as np
import pandas as pd

In [ ]:
# set up our input file path and name
in_file_name = 'words.txt'

In [ ]:
# use pandas to load the data
print('Reading in list of words...')
word_df = pd.read_csv(filepath_or_buffer = in_file_name, sep = ',', header = None)    
# check the first few rows

In [ ]:
# check the first few records
word_df.head()

In [ ]:
# let's specify a a more appropriate column name
col_names = ['word']
word_df.columns = col_names

In [ ]:
# how many words are we working with?
n_words = len(word_df)
print('...found', '{:,}'.format(n_words), 'words to find anagrams for...')

In [ ]:
# convert the only column to a string - just to be safe.
# nan is a word in the dictionary. nan is an internal python value.
word_df['word'] = word_df['word'].astype(np.str)

In [ ]:
# create lower case values of the words
word_df['lcase'] = word_df['word'].str.lower()
# and now drop duplicates
word_df = word_df.drop_duplicates('lcase')

In [ ]:
# 1. find word length
word_df['n_chars'] = word_df['lcase'].str.len()

In [ ]:
# 2. extract the first letter
word_df['first_letter'] = word_df['lcase'].str[:1]

In [ ]:
# 3. Let's aggregate the data to create a new dataframe featuring the counts of words by word length
agg_word_df = word_df['n_chars'].groupby(word_df['n_chars']).agg(np.size).to_frame()
col_names = ['n_words']
agg_word_df.columns = col_names
agg_word_df =  agg_word_df.reset_index()

In [ ]:
agg_word_df.head()
# there are 26 one letter words... that checks out

In [ ]:
# and the tail
agg_word_df.tail()

In [ ]:
# let's do a cross tab - word length by start character
select_columns = ['first_letter', 'n_chars']
ct_word_df = pd.crosstab(index=word_df['first_letter'], columns=word_df['n_chars'])

In [ ]:
# check it
ct_word_df.head()
# looks right

In [ ]:
#  reset the index and write to excel
ct_word_df = ct_word_df.reset_index()

In [ ]:
ct_word_df.head()

In [ ]:
# the amazing thing about pandas is that we can write to disk in a variety of formats.
# Let's pick excel for now.

In [ ]:
# setup the output path
e_file_name = 'words_analysis.xlsx'

In [ ]:
# create the writer object
e_writer = pd.ExcelWriter(e_file_name)

In [ ]:
# let's first write the list of words
word_df.to_excel(excel_writer=e_writer, sheet_name='word_list', index = False)

In [ ]:
# let's write the count of words by character length
agg_word_df.to_excel(excel_writer=e_writer, sheet_name='word_count_by_length', index = False)

In [ ]:
ct_word_df.to_excel(excel_writer=e_writer, sheet_name='word_count_by_length_by_letter', index = False)

In [ ]:
# save and close the excel file
e_writer.save()
e_writer.close()

In [ ]:
word_df.head()

# what if we were trying to find all words by a brute-force technique?

In [ ]:
# let's play with pandas to learn some facts about our list of words and show case how to use pandas
# we're going to make extensive use of the string functions in python and the pandas variates
# pure python: https://docs.python.org/3.6/library/string.html
# pandas versions: https://pandas.pydata.org/pandas-docs/stable/text.html#working-with-text-data

In [ ]:
# what is the maximum character length?
max_char_length = agg_word_df['n_chars'].max()

In [ ]:
max_char_length = 20

In [ ]:
max_char_length

In [ ]:
# how many possible combinations are there?

In [ ]:
n_permutations = math.factorial(max_char_length)

In [ ]:
n_permutations

In [ ]:
# to check all permutations for one 24-character word
# assuming our computer can perform x checks per second
checks_per_second = 100000

In [ ]:
n_seconds = n_permutations / checks_per_second

In [ ]:
print('examining permutations takes', '{:,}'.format(n_seconds), 'seconds...')

In [ ]:
n_minutes = n_seconds / 60 

In [ ]:
print('examining permutations takes', '{:,}'.format(n_minutes), 'minutes...')

In [ ]:
n_hours = n_minutes / 60

In [ ]:
print('examining permutations takes', '{:,}'.format(n_hours), 'hours...')

In [ ]:
n_days = n_hours / 24

In [ ]:
print('examining permutations takes', '{:,}'.format(n_days), 'days...')

In [ ]:
n_years = n_days / 365

In [ ]:
print('examining permutations takes', '{:,}'.format(n_years), 'years...')

In [ ]:
# if we were to check all permutations of all words...
agg_word_df['n_char_factorial'] = agg_word_df['n_chars'].map(math.factorial)

In [ ]:
# and do that for each word...
agg_word_df['n_char_checks'] = agg_word_df['n_words'] * agg_word_df['n_char_factorial']

In [ ]:
total_checks = agg_word_df['n_char_checks'].sum()

In [ ]:
# assuming we can process:
# 1M permutations a second
checks_per_second = 1000000000

In [ ]:
processing_time = total_checks / checks_per_second

In [ ]:
processing_years = processing_time / 60 / 60 / 24 / 365

In [ ]:
print('examining all permutations takes', '{:,}'.format(processing_years), 'years...')

In [ ]:
# that's a long time!

# let's re-think our approach by first focusing on one word: time

In [ ]:
for p in permutations('time'):
    print(p)

In [ ]:
type(p)

In [ ]:
for p in permutations('time'):
    new_word = ''.join(p)
    print(new_word)    

In [ ]:
for p in permutations('time'):    
    new_word = ''.join(sorted(p))
    print(new_word)

In [ ]:
sorted_word_group = 'eimt'

In [ ]:
words_to_examine = ['time', 'mite', 'emit', 'item', 'rite']

In [ ]:
for word in words_to_examine:    
    sorted_word = sorted(word)
    if ''.join(sorted_word) == sorted_word_group:
        print(word, 'is in the', sorted_word_group, 'group')
    else:
        print(word, 'is NOT in the', sorted_word_group, 'group')

In [ ]:
# so we're going to do the same with pandas.
# But we need to write a function to do this. 

def create_sort_word(word):
    sorted_word = sorted(word)
    output_word = ''.join(sorted_word)    
    return output_word

In [ ]:
word_df['word_group'] = word_df['lcase'].map(create_sort_word)

In [ ]:
word_df['sorted_word'] = word_df['lcase'].map(sorted)

In [ ]:
word_df['word_group_2'] = word_df['sorted_word'].map(lambda x: ''.join(x))

In [ ]:
word_df['word_group_3'] = word_df['lcase'].map(sorted).map(lambda x: ''.join(x))

In [ ]:
word_df['word_group_4'] = word_df['lcase'].map(lambda x: ''.join(sorted(x)))

In [ ]:
word_df.head()

In [ ]:
# let's check our work
word_df.head()

In [ ]:
# and the tail
word_df.tail()

In [ ]:
# do a simple group by and sort to count our values
word_df['word_group_count'] = 1
select_columns = ['n_chars', 'word_group', 'word_group_count']
word_group_df = word_df[select_columns].groupby(select_columns[:-1]).agg(np.size)

In [ ]:
# reset the index    
word_group_df = word_group_df.reset_index()  

In [ ]:
word_group_df.head()

In [ ]:
word_group_df.tail()

In [ ]:
len(word_group_df)

In [ ]:
# select only values that occur more than once.
# word_group_df = word_group_df.loc[word_group_df['word_group_count'] > 1, ]

In [ ]:
# very cool.
# So now we pull out the unique word count values, query the df,
# then write to disk.

In [ ]:
n_char_list = word_group_df['n_chars'].unique().tolist()
n_char_list.sort()

In [ ]:
n_char_list[:5]

In [ ]:
# specify the proper line ending when using a csv writer.
output_file_name = 'anagrams_found.txt'

In [ ]:
word_group_df.head()

In [ ]:
s_time = time.time()
# use the csv writer to write this to disk
output_file = open(output_file_name, 'w', newline='')
cw = csv.writer(output_file)    
# intialize some counters
n_anagram_groups = 0
n_anagrams = 0

# enumerate over the word counts
for n_char in n_char_list[:3]:
    curr_df = word_group_df.loc[word_group_df['n_chars']==n_char, ]    
    # sort by the count
    # curr_df = curr_df.sort_values(by='word_group_count')

    # let's do some enumeration   
    word_group_list = curr_df['word_group'].unique().tolist()
    n_word_groups = len(word_group_list)

    print('...found', '{:,}'.format(n_word_groups), 'unique word groups within', n_char, 'digit words.')
    for i_wg, wg in enumerate(word_group_list):
        # the current hash value corresponds to a group of anagrams.
        # get that group of words as a list.
        curr_word_list = word_df.loc[word_df['word_group'] == wg, 'lcase'].tolist()
        # print(wg)
        # print(curr_word_list)

        cw.writerow(curr_word_list)
        # increment the counters to find the total number of anagram groups
        # and words
        n_anagram_groups += 1
        n_anagrams += len(curr_word_list)

# close my file
output_file.close()
e_time = time.time()

In [ ]:
p_time = e_time - s_time

In [ ]:
print('...finding anagrams took', '{:,}'.format(round(p_time,1)), 'seconds...')

In [ ]:
# how many anagram groups and anagrams did we find?
n_anagram_groups = '{:,}'.format(n_anagram_groups)
n_anagrams = '{:,}'.format(n_anagrams)
print('...found', n_anagram_groups, 'anagram groups consisting of',
      n_anagrams, 'words...')